In [7]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:

#num_drugs = ~3200
#num_samples = ~24000
#num_moa = 206

#read features dataset
train_df = pd.read_csv('train_features.csv')
test_df = pd.read_csv('test_features.csv')


print('number of samples: {}'.format(len(train_df)))
#read targets


number of samples: 23814


In [10]:
#transform qualitative datapoints
#cp_type: 0 = control, 1 = treatment
#cp_dose: 0 = D1, 1 = D2
#cp_time: no need to transform

train_df['cp_type'].replace(['ctl_vehicle', 'trt_cp'], [0,1], inplace=True)
test_df['cp_type'].replace(['ctl_vehicle', 'trt_cp'], [0,1], inplace=True)

train_df['cp_dose'].replace(['D1','D2'],[0,1],inplace = True)
test_df['cp_dose'].replace(['D1','D2'],[0,1],inplace = True)

In [11]:
#create dataframes of gene exp and viability features

gene_exp_columns = [i for i in train_df.columns if 'g-' in i]
gene_exp_df = train_df[gene_exp_columns]

cell_viability_columns = [j for j in train_df.columns if 'c-' in j]
cell_viability_df = train_df[cell_viability_columns]

#index treatment vs control sig_ids



In [29]:
target_df = pd.read_csv('train_targets_scored.csv')
print('number of MoA: {}'.format(len(target_df.columns)-1))

moa_list = list(target_df.columns[1:]) #drop sig_id

#make sure all treatment sig_ids have one MoA and all control sig_ids have none
ctrl_indices = train_df.index[train_df['cp_type'] == 0].tolist()
trt_indices = train_df.index[train_df['cp_type'] == 1].tolist()

abnormal_ctrl_sig_id = []
for i in ctrl_indices:
    row_sum = np.sum(target_df.iloc[i].values[1:])
    if row_sum > 0:
        abnormal_ctrl_sig_id.append(i,row_sum)

abnormal_trt_sig_id = []
for j in trt_indices:
    row_sum = np.sum(target_df.iloc[j].values[1:])
    if row_sum != 1:
        abnormal_trt_sig_id.append((j,row_sum))

#there are multiple treatment subjects with either zero or multiple mechanism of actions, need to see about what drugs they take to be ommitted. 


number of MoA: 206


In [33]:
#test to see how many drugs are in each mechanism of action
#'          ' how many subjects per drug

drug_df = pd.read_csv('train_drug.csv')

num_drugs = len(np.unique(drug_df['drug_id']))
print('number of drugs: {}'.format(num_drugs))
drug_list = set(drug_df['drug_id'])
#find number of subjects per drug

drug_dict = {}
subj_dist = []
for drug in drug_list:
    num_subjects = len(drug_df.index[drug_df['drug_id'] == drug].tolist())
    subj_dist.append(num_subjects)
    drug_dict.update({drug: num_subjects})

print('number of subjects per given drug: ')
print(np.sort(list(drug_dict.values()))[:24:-1]) #can see some drugs are given to many subjects 

number of drugs: 3289
number of subjects per given drug: 
[1866  718  246 ...    1    1    1]


In [38]:
#number of subjects per mechanism of action

moa_dict = {}
for moa in list(target_df.columns[1:]): #exclude sig_id
    num_subj = target_df[target_df[moa] == 1].shape[0]
    moa_dict.update({moa: num_subj})

In [45]:
sorted(((v,k) for k,v in moa_dict.items()), reverse=False)

[(1, 'atp-sensitive_potassium_channel_antagonist'),
 (1, 'erbb2_inhibitor'),
 (6, 'antiarrhythmic'),
 (6, 'atm_kinase_inhibitor'),
 (6, 'autotaxin_inhibitor'),
 (6, 'calcineurin_inhibitor'),
 (6, 'coagulation_factor_inhibitor'),
 (6, 'diuretic'),
 (6, 'elastase_inhibitor'),
 (6, 'laxative'),
 (6, 'leukotriene_inhibitor'),
 (6, 'lxr_agonist'),
 (6, 'nicotinic_receptor_agonist'),
 (6, 'protein_phosphatase_inhibitor'),
 (6, 'retinoid_receptor_antagonist'),
 (6, 'steroid'),
 (6, 'tropomyosin_receptor_kinase_inhibitor'),
 (6, 'ubiquitin_specific_protease_inhibitor'),
 (7, 'aldehyde_dehydrogenase_inhibitor'),
 (7, 'bacterial_membrane_integrity_inhibitor'),
 (7, 'norepinephrine_reuptake_inhibitor'),
 (7, 'tlr_antagonist'),
 (12, 'adenylyl_cyclase_activator'),
 (12, 'ampk_activator'),
 (12, 'analgesic'),
 (12, 'anticonvulsant'),
 (12, 'antihistamine'),
 (12, 'atp_synthase_inhibitor'),
 (12, 'catechol_o_methyltransferase_inhibitor'),
 (12, 'lipase_inhibitor'),
 (12, 'monoacylglycerol_lipase_inh